In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
# from sklearn import qda, lda
import math
from numpy.linalg import inv
import lda
import qda
import lr
import htwt

In [2]:
X, y = htwt.get()

In [3]:
model = lr.fit(X, y)
np.mean(lr.predict(model, X) == y)

0.87142857142857144

In [4]:
model = qda.fit(X, y)
np.mean(qda.predict(model, X) == y)

0.88095238095238093

In [6]:
X = (X - np.mean(X, axis = 0)) / np.std(X, axis = 0)
model= lda.fit(X, y)
np.mean(lda.predict(model, X) == y)

0.87142857142857144